# Data verification and cleaning

## Importing taxi rides data

In [1]:
import pandas as pd
import os
import sys

In [2]:
data_location_rides = "../data/yellow_tripdata_2022-01.parquet"
df = pd.read_parquet(data_location_rides)

In [3]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [4]:
# Add index column 
tripID = range(len(df))
df["tripID"] = tripID

In [5]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,tripID
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0,0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0,1
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0,2
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0,3
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0,4


In [6]:
# Check if data is sufficiently connected
df["PULocationID"].value_counts()

237    121630
236    120814
132    103485
161     88237
186     80580
        ...  
105         1
172         1
176         1
27          1
187         1
Name: PULocationID, Length: 257, dtype: int64

## Generating drivers 

In [7]:
import names
import random
import string

In [8]:
avg_nb_of_rides_per_driver = 500
nb_drivers = len(df) // avg_nb_of_rides_per_driver

In [9]:
# Generating names
drivers_first_names = [names.get_first_name() for i in range(nb_drivers)]
drivers_last_names = [names.get_last_name() for i in range(nb_drivers)]

In [10]:
# Generating licence plate number
def plate_number_gen():
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=6))

plates_numbers = [plate_number_gen() for i in range(nb_drivers)]

In [11]:
drivers_data = {"driver_id" : range(nb_drivers),
                "last_name" : drivers_last_names, 
                "first_name" : drivers_first_names, 
                "plate_number" : plates_numbers}

In [12]:
df_drivers = pd.DataFrame(drivers_data)

In [13]:
df_drivers.head()

,driver_id,last_name,first_name,plate_number
0,0,Phipps,Faye,L2VMFM
1,1,Terrell,Danette,DG429Q
2,2,Shelton,Randall,2MWOFQ
3,3,Charpia,Lissette,2DTIWR
4,4,Mischler,George,OQE8LY


In [14]:
# Truncate original dataframe to have len(df_drivers) * avg_nb_of_rides_per_driver rows
df_rides = df[:len(df_drivers) * avg_nb_of_rides_per_driver]

In [15]:
# Creating new column in df_rides to add the driver
from itertools import repeat

driver_id = pd.Series([x for item in drivers_data["driver_id"] 
                        for x in repeat(item, avg_nb_of_rides_per_driver)], index = range(len(df_rides)))

df_rides = df_rides.assign(driverID = driver_id.values)

In [16]:
df_rides

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,tripID,driverID
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,...,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0,0,0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,...,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0,1,0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,...,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0,2,0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,...,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0,3,0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,...,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2463495,2,2022-01-31 19:25:00,2022-01-31 19:55:00,NaN,8.40,NaN,None,34,198,0,...,0.0,0.5,5.64,0.0,0.3,31.49,NaN,NaN,2463495,4926
2463496,2,2022-01-31 19:41:00,2022-01-31 19:53:00,NaN,1.85,NaN,None,49,61,0,...,0.0,0.5,1.19,0.0,0.3,12.41,NaN,NaN,2463496,4926
2463497,2,2022-01-31 19:35:00,2022-01-31 19:49:00,NaN,2.36,NaN,None,113,45,0,...,0.0,0.5,3.39,0.0,0.3,19.16,NaN,NaN,2463497,4926
2463498,1,2022-01-31 19:02:18,2022-01-31 19:05:27,NaN,0.60,NaN,None,229,233,0,...,1.0,0.5,1.24,0.0,0.3,9.54,NaN,NaN,2463498,4926


In [17]:
df_rides["payment_type"].value_counts()

1    1874874
2     495171
0      71072
3      11709
4      10673
5          1
Name: payment_type, dtype: int64

## Creating csv files

In [19]:
import os
   
path_rides = '../data/rides.csv'
path_drivers = '../data/drivers.csv'
   
if(not os.path.exists(path_rides)):
    df_rides.to_csv('../data/rides.csv')

if(not os.path.exists(path_drivers)):
    df_drivers.to_csv('../data/drivers.csv')


In [20]:
df_test = df_rides[3000:13000]
df_test.to_csv("../data/rides_test2.csv")